In [1]:
import pandas as pd
import numpy as np
import ast
import os
import re
import openpyxl

from mvp.process import *
from mvp.eval_utils import *

# 결과파일 위치지정

In [66]:
method = 'mvp'
task = 'asqp'
dataset = 'rest15'
source_file = 'dev'
output_file = 'result_asqp_rest15_dev_asqp_rest16_only_path1_beam1'
save_file = 'as15_dev_asqp_rest16_only'

home = os.path.expanduser('~/')
data_dir = os.path.join(home, f'ABSA/data/{task}/{dataset}')

if method == 'mvp':
    output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/top_5_post_data1.0')
elif method == 'dlo':
    output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/top_5_post_data1.0')
elif method == 'paraphrase':
    output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/post_data1.0')

data_path = os.path.join(data_dir, f'{source_file}.txt')
output_path = os.path.join(output_dir,f'{output_file}.pickle')
save_path = os.path.join(output_dir,f'{save_file}.xlsx')

print('data_path: ',data_path)
print('output_path: ',output_path)
print('save_path: ',save_path)

data_path:  /home/elicer/ABSA/data/asqp/rest15/dev.txt
output_path:  /home/elicer/ABSA/outputs/mvp/asqp/rest15/top_5_post_data1.0/result_asqp_rest15_dev_asqp_rest16_only_path1_beam1.pickle
save_path:  /home/elicer/ABSA/outputs/mvp/asqp/rest15/top_5_post_data1.0/as15_dev_asqp_rest16_only.xlsx


In [67]:
with open(output_path, 'rb') as f:
    loaded_object = pd.read_pickle(f)

predict = extract_quad(loaded_object[0],'pred')
predict_df = trans_tuple(predict)
columns_to_rename = {col: col + '_p' for col in predict_df.columns if col != 'sent_id'}
predict_df.rename(columns=columns_to_rename, inplace=True)

target = extract_quad(loaded_object[1],'gold')
target_df = trans_tuple(target)
columns_to_rename = {col: col + '_t' for col in target_df.columns if col != 'sent_id'}
target_df.rename(columns=columns_to_rename, inplace=True)

scores = compute_f1_scores(predict, target, verbose=True)
exp_results = "{} ==>> precision: {:.2f} recall: {:.2f} F1 = {:.2f}".format(
    output_file, scores['precision'], scores['recall'], scores['f1'])
print(exp_results)
print()

df = error_type(predict_df,target_df)
df['match'] = np.where(
    df['quad_ord_t'].notna() & df['quad_ord_p'].notna(), 'match',
    np.where(df['quad_ord_t'].isna() & df['quad_ord_p'].notna(), 'over prediction',
             np.where(df['quad_ord_t'].notna() & df['quad_ord_p'].isna(), 'missing quad', 'other')
            )
)
df['A_err'] = df['score'].str[0]
df['C_err'] = df['score'].str[1]
df['O_err'] = df['score'].str[2]
df['S_err'] = df['score'].str[3]

inputs, _ = split_read(data_path)
s_df = pd.DataFrame({'sent_id': [index + 1 for index, value in enumerate(inputs)],'sentence': inputs})
result = pd.merge(df, s_df, on = 'sent_id', how='left')

result.to_excel(save_path)

FileNotFoundError: [Errno 2] No such file or directory: '/home/elicer/ABSA/outputs/mvp/asqp/rest15/top_5_post_data1.0/result_asqp_rest15_dev_asqp_rest16_only_path1_beam1.pickle'

In [49]:
method = 'mvp'
task = 'asqp'
dataset = 'rest15'

home = os.path.expanduser('~/')
data_dir = os.path.join(home, f'ABSA/data/{task}/{dataset}')

if method == 'mvp':
    output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/top_5_post_data1.0')
elif method == 'dlo':
    output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/top_5_post_data1.0')
elif method == 'paraphrase':
    output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/post_data1.0')

print('data_dir: ',data_dir)
print('output_dir: ',output_dir)

data_dir:  /home/elicer/ABSA/data/asqp/rest15
output_dir:  /home/elicer/ABSA/outputs/mvp/asqp/rest15/top_5_post_data1.0


In [53]:
for i in range(1, 11):  # 1부터 10까지의 파일을 생성

    source_file = f'zero_train_asqp_rest16_{i}.txt'
    output_file = f'result_asqp_rest15_zero_train_asqp_rest16_{i}_path1_beam1.pickle'
    save_path = f'as15_train_asqp_rest16_{i}.xlsx'
    
    data_path = os.path.join(data_dir, source_file)
    output_path = os.path.join(output_dir, output_file)
    save_path = os.path.join(output_dir, save_path)

    with open(output_path, 'rb') as f:
        loaded_object = pd.read_pickle(f)
    
    predict = extract_quad(loaded_object[0],'pred')
    predict_df = trans_tuple(predict)
    columns_to_rename = {col: col + '_p' for col in predict_df.columns if col != 'sent_id'}
    predict_df.rename(columns=columns_to_rename, inplace=True)
    
    target = extract_quad(loaded_object[1],'gold')
    target_df = trans_tuple(target)
    columns_to_rename = {col: col + '_t' for col in target_df.columns if col != 'sent_id'}
    target_df.rename(columns=columns_to_rename, inplace=True)
    
    scores = compute_f1_scores(predict, target, verbose=True)
    exp_results = "{} ==>> precision: {:.2f} recall: {:.2f} F1 = {:.2f}".format(
        output_file, scores['precision'], scores['recall'], scores['f1'])
    print(exp_results)
    print()
    
    df = error_type(predict_df,target_df)
    df['match'] = np.where(
        df['quad_ord_t'].notna() & df['quad_ord_p'].notna(), 'match',
        np.where(df['quad_ord_t'].isna() & df['quad_ord_p'].notna(), 'over prediction',
                 np.where(df['quad_ord_t'].notna() & df['quad_ord_p'].isna(), 'missing quad', 'other')
                )
    )
    df['A_err'] = df['score'].str[0]
    df['C_err'] = df['score'].str[1]
    df['O_err'] = df['score'].str[2]
    df['S_err'] = df['score'].str[3]
    
    inputs, _ = split_read(data_path)
    s_df = pd.DataFrame({'sent_id': [index + 1 for index, value in enumerate(inputs)],'sentence': inputs})
    result = pd.merge(df, s_df, on = 'sent_id', how='left')
    
    result.to_excel(save_path)

number of gold spans: 934, predicted spans: 936, hit: 448
result_asqp_rest15_zero_train_asqp_rest16_1_path1_beam1.pickle ==>> precision: 47.86 recall: 47.97 F1 = 47.91

init: target= 934  predict= 936
ACOS: 445  + target 489  =  934   :: predict= 492
AOS_gold: 73  + target 416  =  489   :: predict= 421
COS_gold: 42  + target 374  =  416   :: predict= 380
ACS_gold: 107  + target 267  =  374   :: predict= 274
ACO_gold: 19  + target 248  =  267   :: predict= 255
AO_gold: 4  + target 244  =  248   :: predict= 251
AC_gold: 21  + target 223  =  244   :: predict= 230
AS_gold: 43  + target 180  =  223   :: predict= 189
CO_gold: 6  + target 174  =  180   :: predict= 183
CS_gold: 30  + target 144  =  174   :: predict= 155
OS_gold: 29  + target 115  =  144   :: predict= 127
A_gold: 6  + target 109  =  115   :: predict= 121
O_gold: 2  + target 107  =  109   :: predict= 119
C_gold: 7  + target 100  =  107   :: predict= 112
S_gold: 17  + target 83  =  100   :: predict= 95
Data read. Total count:  60

In [15]:
data_name = 'train_asqp_rest16_only_n10'
data_path = f'/home/elicer/ABSA/data/asqp/rest15/{data_name}.txt'
output_path = f'/home/elicer/ABSA/outputs/mvp/asqp/rest15/top_5_post_data1.0/result_asqp_rest15_{data_name}_path1_beam1.pickle'
save_path = f'/home/elicer/ABSA/outputs/mvp/asqp/rest15/top_5_post_data1.0/{data_name}.xlsx'

with open(output_path, 'rb') as f:
    loaded_object = pd.read_pickle(f)

predict = extract_quad(loaded_object[0],'pred')
predict_df = trans_tuple(predict)
columns_to_rename = {col: col + '_p' for col in predict_df.columns if col != 'sent_id'}
predict_df.rename(columns=columns_to_rename, inplace=True)

target = extract_quad(loaded_object[1],'gold')
target_df = trans_tuple(target)
columns_to_rename = {col: col + '_t' for col in target_df.columns if col != 'sent_id'}
target_df.rename(columns=columns_to_rename, inplace=True)

scores = compute_f1_scores(predict, target, verbose=True)
exp_results = "{} ==>> precision: {:.2f} recall: {:.2f} F1 = {:.2f}".format(
    data_name, scores['precision'], scores['recall'], scores['f1'])
print(exp_results)
print()

df = error_type(predict_df,target_df)
df['match'] = np.where(
    df['quad_ord_t'].notna() & df['quad_ord_p'].notna(), 'match',
    np.where(df['quad_ord_t'].isna() & df['quad_ord_p'].notna(), 'over prediction',
             np.where(df['quad_ord_t'].notna() & df['quad_ord_p'].isna(), 'missing quad', 'other')
            )
)
df['A_err'] = df['score'].str[0]
df['C_err'] = df['score'].str[1]
df['O_err'] = df['score'].str[2]
df['S_err'] = df['score'].str[3]

inputs, _ = split_read(data_path)
s_df = pd.DataFrame({'sent_id': [index + 1 for index, value in enumerate(inputs)],'sentence': inputs})

result = pd.merge(df, s_df, on = 'sent_id', how='left')

# 결과 데이터셑 저장
result.to_excel(save_path)


number of gold spans: 9340, predicted spans: 9181, hit: 4040
train_asqp_rest16_only_n10 ==>> precision: 44.00 recall: 43.25 F1 = 43.63

init: target= 9340  predict= 9181
ACOS: 4030  + target 5310  =  9340   :: predict= 5161
AOS_gold: 727  + target 4583  =  5310   :: predict= 4455
COS_gold: 411  + target 4172  =  4583   :: predict= 4054
ACS_gold: 1289  + target 2883  =  4172   :: predict= 2794
ACO_gold: 201  + target 2682  =  2883   :: predict= 2593
AO_gold: 37  + target 2645  =  2682   :: predict= 2556
AC_gold: 183  + target 2462  =  2645   :: predict= 2373
AS_gold: 448  + target 2014  =  2462   :: predict= 1944
CO_gold: 60  + target 1954  =  2014   :: predict= 1884
CS_gold: 317  + target 1637  =  1954   :: predict= 1578
OS_gold: 258  + target 1379  =  1637   :: predict= 1334
A_gold: 66  + target 1313  =  1379   :: predict= 1268
O_gold: 16  + target 1297  =  1313   :: predict= 1252
C_gold: 92  + target 1205  =  1297   :: predict= 1164
S_gold: 251  + target 954  =  1205   :: predict= 92